This workshop will cover some of the genomic annotation packages from Bioconductor and how to use them. This information might be helpful if you want to do things like:   
- Convert between different gene identifiers or chromosome naming conventions       
- Fine gene promoter region ranges or sequences     
- Figure out which GO terms are associated with your genes of interest          

Let's load some packages:

In [5]:
library("tidyverse")
library("ggplot2")
library("BiocManager")
library("gridExtra")
library("airway")
library("AnnotationHub")
library("clusterProfiler")
library("enrichplot")
library("biomaRt")
library("DESeq2")
library("GenomicFeatures")
library("enrichplot")
library("BSgenome")
library("BSgenome.Hsapiens.UCSC.hg19")

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Bioconductor version '3.16' is out-of-date; the current release version '3.17'
  is available with R version '4.3'; see https://bioconductor.org/install


Attaching package: ‘gridExtra’


The following object is masked from ‘packa

Lets import some data to work with -- `res` is the `results` object and `rld` is the rlog transformed counts from DESeq2 differential expression analysis run on the `airway` data. We are comparing the dexamethasone treatment conditions, comparing treated to untreated.


In [6]:
res <- readRDS("res.rds")

In [7]:
res

log2 fold change (MAP): dex trt vs untrt 
Wald test p-value: dex trt vs untrt 
DataFrame with 29391 rows and 5 columns
                  baseMean log2FoldChange     lfcSE      pvalue       padj
                 <numeric>      <numeric> <numeric>   <numeric>  <numeric>
ENSG00000000003 708.602170     -0.3584541  0.100036 0.000152017 0.00128122
ENSG00000000419 520.297901      0.1860726  0.108183 0.065337210 0.19620790
ENSG00000000457 237.163037      0.0313468  0.130347 0.791504963 0.91119406
ENSG00000000460  57.932633     -0.0472622  0.212589 0.758803336 0.89463420
ENSG00000000938   0.318098     -0.0123431  0.309182 0.693732273         NA
...                    ...            ...       ...         ...        ...
ENSG00000273485   1.286448    -0.00445615  0.303747    0.936702         NA
ENSG00000273486  15.452537    -0.04309283  0.264183    0.756289   0.893661
ENSG00000273487   8.163235     0.19116398  0.355670    0.134373   0.329057
ENSG00000273488   8.584479     0.02194011  0.279542    0

In [8]:
rld <- readRDS("rld.rds")

In [9]:
rld

class: DESeqTransform 
dim: 29391 8 
metadata(2): '' version
assays(1): ''
rownames(29391): ENSG00000000003 ENSG00000000419 ... ENSG00000273488
  ENSG00000273489
rowData names(35): baseMean baseVar ... maxCooks rlogIntercept
colnames(8): SRR1039508 SRR1039509 ... SRR1039520 SRR1039521
colData names(10): SampleName cell ... BioSample sizeFactor

Let's work with the `res` results table. We can see that each row is a gene (`ENSG...`) and each column gives us some information about the differential expression analysis. These gene IDs are not particularly informative, but we can use biomaRt to fix that.

# Using biomaRt

The [biomaRt](https://bioconductor.org/packages/release/bioc/html/biomaRt.html) package makes it easy to query public repositories of biological data. We can use biomaRt to query Ensembl for annotations so that we can look for 'housekeeping genes' which are typically considered to be stably expressed and shouldn't show large variations across different samples. We have selected a list of genes based on two publications that queried public cancer genome data to find housekeeping genes for use with RNA-seq from cancer cell lines (https://doi.org/10.1186/s12859-019-2809-2, https://doi.org/10.3389/fgene.2019.00097). 

First, let's load biomaRt and make a vector of the gene symbols from the published data:

In [10]:
housekeeping <- c('PCBP1','RER1', 'RPN1', 'PUM1', 'IPO8')

Then we can see what BioMarts are available:

In [11]:
listMarts()

biomart,version
<chr>,<chr>
ENSEMBL_MART_ENSEMBL,Ensembl Genes 109
ENSEMBL_MART_MOUSE,Mouse strains 109
ENSEMBL_MART_SNP,Ensembl Variation 109
ENSEMBL_MART_FUNCGEN,Ensembl Regulation 109


Let's use `ENSEMBL_MART_ENSEMBL` (you might get an error that says `Ensembl site unresponsive, trying uswest mirror`, run `?useEnsembl` to get more information about available options).

In [12]:
ensembl <- useEnsembl(biomart = 'ENSEMBL_MART_ENSEMBL')

Ensembl site unresponsive, trying www mirror



You can see a list of all available datasets within the mart if you run `listDatasets(ensembl)` -- there are many (~200 of them), so let's narrow it down a little and look only for human data.

In [13]:
searchDatasets(mart = ensembl, pattern = 'hsapiens')

ERROR: Error in curl::curl_fetch_memory(url, handle = handle): Timeout was reached: [www.ensembl.org:443] Operation timed out after 10001 milliseconds with 0 bytes received


Now we can put it all together to create a BioMart object: (you might get an error that says `Ensembl site unresponsive, trying uswest mirror`)

In [ ]:
ensembl <- useEnsembl(biomart = 'ENSEMBL_MART_ENSEMBL', dataset='hsapiens_gene_ensembl', mirror = 'uswest')

Later, we will use the `getBM()` function to query BioMart (this is the main function of biomaRt). This function takes the followingarguments:

`attributes`: the attributes you want to retrieve                     
`filters`: the filters that should be used in the query                    
`values`: the values of the filters                    
`mart`: the mart object you want to use.   

We can use the `listAttributes` function to see what information is available in `ensembl` (limiting it here to the first 5)

In [ ]:
attributes = listAttributes(ensembl)
attributes[1:5,]

Note that there are ~3000 attributes for this mart! We only care a about two -- `ensembl_gene_id` and `hgnc_symbol`.

We can use the `listFilters` function to see what our filtering options are (limiting it here to the first 5)

In [ ]:
filters = listFilters(ensembl)
filters[1:5,]

We can use `getBM` to query the BioMart object                   

In [ ]:
ensembl_bm <- getBM(
    attributes = c('ensembl_gene_id','hgnc_symbol'),
    filters = 'hgnc_symbol',
    values = housekeeping, 
    mart = ensembl)
ensembl_bm

Let's look at the `rlog` normalized counts for our housekeeping genes:

In [ ]:
housekeeping_rld <- data.frame(assay(rld)[ensembl_bm$ensembl_gene_id, ])
head(housekeeping_rld)

The `ensembl_gene_id` is currently stored as the rownames. Let's go ahead and turn it into a column in the data frame:

In [ ]:
housekeeping_rld$ensembl_gene_id <- rownames(housekeeping_rld)
head(housekeeping_rld)

Then we use the `gather` function to convert the data to a long format.

In [ ]:
housekeeping_rld_tidy <- gather(housekeeping_rld, key = 'sample', value = 'rlog_counts', SRR1039508:SRR1039521)
head(housekeeping_rld_tidy)

Let's add the annotation information we pulled from biomaRt:

In [ ]:
housekeeping_rld_tidy <- inner_join(ensembl_bm, housekeeping_rld_tidy, by = 'ensembl_gene_id')
head(housekeeping_rld_tidy)

Let's look at the expression of our housekeeping genes to see if they look stably expressed in our data:

In [ ]:
options(repr.plot.width=10, repr.plot.height=5)

ggplot(housekeeping_rld_tidy, aes(x=sample, y=rlog_counts)) + 
geom_bar(stat="identity") +
facet_wrap(~hgnc_symbol, nrow = 1) +
theme(axis.text.x = element_text(angle = 90))

These housekeeping genes look stably expressed across each sample.

# Using AnnotationHub

Now we can try using AnnotationHub to do something similar to what we just did with `biomaRt`.
Many of the data types we will work with from AnnotationHub are based on the `AnnotationDb` object class -- including OrgDb, TxDb, and many others. This means that they have many functions and methods in common (http://web.mit.edu/~r/current/arch/i386_linux26/lib/R/library/AnnotationDbi/html/AnnotationDb-class.html).               

First, let's connect to the hub using `AnnotationHub` and look at the output.

In [21]:
ah <- AnnotationHub()
ah

snapshotDate(): 2022-10-31



AnnotationHub with 69797 records
# snapshotDate(): 2022-10-31
# $dataprovider: Ensembl, BroadInstitute, UCSC, ftp://ftp.ncbi.nlm.nih.gov/g...
# $species: Homo sapiens, Mus musculus, Drosophila melanogaster, Bos taurus,...
# $rdataclass: GRanges, TwoBitFile, BigWigFile, EnsDb, Rle, OrgDb, ChainFile...
# additional mcols(): taxonomyid, genome, description,
#   coordinate_1_based, maintainer, rdatadateadded, preparerclass, tags,
#   rdatapath, sourceurl, sourcetype 
# retrieve records with, e.g., 'object[["AH5012"]]' 

            
  AH5012   |
  AH5013   |
  AH5014   |
  AH5015   |
  AH5016   |
  ...       
  AH111329 |
  AH111330 |
  AH111331 |
  AH111332 |
  AH111334 |
           title                                                              
  AH5012   Chromosome Band                                                    
  AH5013   STS Markers                                                        
  AH5014   FISH Clones                                                        
  AH50

You might get an error:

Error in AnnotationHub(): DEFUNCT: As of AnnotationHub (>2.23.2), default caching location has changed.
  Problematic cache: /users/jwalla12/.cache/AnnotationHub
  See https://bioconductor.org/packages/devel/bioc/vignettes/AnnotationHub/inst/doc/TroubleshootingTheCache.html#default-caching-location-update

Traceback:

1. AnnotationHub()
2. stop(msg = paste0("DEFUNCT: As of AnnotationHub (>2.23.2), default caching location has changed.\n", 
 .     "  Problematic cache: ", path.expand(olddefault), "\n", "  See https://bioconductor.org/packages/devel/bioc/vignettes/AnnotationHub/inst/doc/TroubleshootingTheCache.html#default-caching-location-update\n"))
 


If you do, run this and then re-run the code to connect to AnnotationHub:

moveFiles<-function(package){
        olddir <- path.expand(rappdirs::user_cache_dir(appname=package))
        newdir <- tools::R_user_dir(package, which="cache")
        dir.create(path=newdir, recursive=TRUE)
        files <- list.files(olddir, full.names =TRUE)
        moveres <- vapply(files,
        FUN=function(fl){
          filename = basename(fl)
          newname = file.path(newdir, filename)
          file.rename(fl, newname)
        },
        FUN.VALUE = logical(1))
        if(all(moveres)) unlink(olddir, recursive=TRUE)
    }

package="AnnotationHub"
moveFiles(package)

This is one of the very nice things about using AnnotationHub -- there's many data providers, data classes, and organisms represented in the hub. You can access these elements using `$` accessor:

In [22]:
head(unique(ah$dataprovider))
length(unique(ah$dataprovider))

[1] "UCSC"        "Ensembl"     "RefNet"      "Inparanoid8" "NHLBI"      
[6] "ChEA"

[1] 79

In [23]:
unique(ah$rdataclass)


[1] "GRanges"                           "data.frame"                       
 [3] "Inparanoid8Db"                     "TwoBitFile"                       
 [5] "ChainFile"                         "SQLiteConnection"                 
 [7] "biopax"                            "BigWigFile"                       
 [9] "AAStringSet"                       "MSnSet"                           
[11] "mzRident"                          "list"                             
[13] "TxDb"                              "Rle"                              
[15] "EnsDb"                             "VcfFile"                          
[17] "igraph"                            "data.frame, DNAStringSet, GRanges"
[19] "sqlite"                            "data.table"                       
[21] "character"                         "SQLite"                           
[23] "SQLiteFile"                        "Tibble"                           
[25] "Rda"                               "FaFile"                           
[27] "String"                            "CompDb"                           
[29] "OrgDb"

## OrgDb objects

One of the options you can see here is `OrgDb`, which is an organism-specific, genome wide annotation. We can use it to map between different gene ID types using a central identifier (usually Entrez gene ID). 

OrgDb names are always of the form: org.Ab.id.db (e.g.  org.Sc.sgd.db) where Ab is a 2-letter abbreviation of the organism and id is an abbreviation (in lower-case) describing the type of central identifier (`eg` for Entrez Gene ids).

Let's see what our options are for `Homo sapiens` and `OrgDb`:

In [24]:
AnnotationHub::query(ah, pattern = c("Homo sapiens", "OrgDb"))

AnnotationHub with 1 record
# snapshotDate(): 2022-10-31
# names(): AH107059
# $dataprovider: ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/
# $species: Homo sapiens
# $rdataclass: OrgDb
# $rdatadateadded: 2022-09-29
# $title: org.Hs.eg.db.sqlite
# $description: NCBI gene ID based annotations about Homo sapiens
# $taxonomyid: 9606
# $genome: NCBI genomes
# $sourcetype: NCBI/ensembl
# $sourceurl: ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/, ftp://ftp.ensembl.org/p...
# $sourcesize: NA
# $tags: c("NCBI", "Gene", "Annotation") 
# retrieve record with 'object[["AH107059"]]' 

So you can see here that there is an OrgDb for Homo sapiens that uses Entrez gene ID as the central identifier.

In [25]:
orgdb <- AnnotationHub::query(ah, pattern = c("Homo sapiens", "OrgDb"))[[1]]

downloading 1 resources

retrieving 1 resource

loading from cache

Warning message:
“Corrupt Cache: resource path
  See AnnotationHub's TroubleshootingTheCache vignette section on corrupt cache
  cache: /users/jwalla12/.cache/R/AnnotationHub
  potential duplicate files: 
    1584245bff5a0_105758
    1c9da3b39ddca_105758
    41c039f9905d_113805
    876f6f9145d2_113805
    2eb7f73f7864b_58996
    41c055f6ecc7_58996
    3f638285427db_90869
    a37a3b8dc6e2_90869
Continuing with first found cached file”


In [26]:
orgdb


Please see: help('select') for usage information



OrgDb object:
| DBSCHEMAVERSION: 2.1
| Db type: OrgDb
| Supporting package: AnnotationDbi
| DBSCHEMA: HUMAN_DB
| ORGANISM: Homo sapiens
| SPECIES: Human
| EGSOURCEDATE: 2022-Sep12
| EGSOURCENAME: Entrez Gene
| EGSOURCEURL: ftp://ftp.ncbi.nlm.nih.gov/gene/DATA
| CENTRALID: EG
| TAXID: 9606
| GOSOURCENAME: Gene Ontology
| GOSOURCEURL: http://current.geneontology.org/ontology/go-basic.obo
| GOSOURCEDATE: 2022-07-01
| GOEGSOURCEDATE: 2022-Sep12
| GOEGSOURCENAME: Entrez Gene
| GOEGSOURCEURL: ftp://ftp.ncbi.nlm.nih.gov/gene/DATA
| KEGGSOURCENAME: KEGG GENOME
| KEGGSOURCEURL: ftp://ftp.genome.jp/pub/kegg/genomes
| KEGGSOURCEDATE: 2011-Mar15
| GPSOURCENAME: UCSC Genome Bioinformatics (Homo sapiens)
| GPSOURCEURL: 
| GPSOURCEDATE: 2022-Aug31
| ENSOURCEDATE: 2022-Jun28
| ENSOURCENAME: Ensembl
| ENSOURCEURL: ftp://ftp.ensembl.org/pub/current_fasta
| UPSOURCENAME: Uniprot
| UPSOURCEURL: http://www.UniProt.org/
| UPSOURCEDATE: Fri Sep 23 16:26:35 2022

What types of data can we retrieve from the OrgDb? Let's use `keytypes()` to find out. 

The likely use case is that you are hoping to convert between different ID types (like we did with biomaRt). One way to do this is the `select()` function. AnnotationHub imports this function from AnnotationDbi so you can run `?AnnotationDbi::select` to view the help. As I said before, OrgDbs are based on the AnnotationDb object base class and the `select`, `columns`, `keys`, and `keytypes` arguments are used together to query AnnotationDb objects.

`select` will retrieve the data as a data.frame based on parameters for selected keys columns and keytype arguments.

`columns` shows which kinds of data can be returned for the AnnotationDb object.

`keys` returns keys for the database contained in the AnnotationDb object. 

`keytypes` allows the user to discover which keytypes can be passed in to select or keys and the keytype argument.

We can view columns and keytypes -- note that these can be the same but are not always the same.

In [16]:
columns(orgdb)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'columns': object 'orgdb' not found


In [ ]:
keytypes(orgdb)

Let's look at a few examples of what the key entries look like

In [ ]:
head(keys(orgdb, keytype="SYMBOL"))

We can try running `select` to look for the housekeeping genes in the OrgDb to retrieve their ENSEMBL and ENTREZIDs:

In [ ]:
ens_entr_orgdb <- select(orgdb, keys=housekeeping, 
       columns=c("ENSEMBL","ENTREZID"), 
       keytype="SYMBOL")
ens_entr_orgdb

As you can see, this returned a 1:1 mapping between keys and columns, but this might not always be the case. What happens if we use "GO" as one of the columns?

In [ ]:
go_orgdb <- select(orgdb, keys=housekeeping, 
       columns=c("ENSEMBL","GO"), 
       keytype="SYMBOL")
head(go_orgdb)

This might not be the ideal outcome for you. Another approach is to use the `mapIds` function. `mapIds` is similar to `select` in that it uses `keys` and `keytypes` but it uses `column` instead of `columns` and can only return one column type, 

In [ ]:
mapped_go <- mapIds(orgdb, keys=housekeeping, 
       column="GO", 
       keytype="SYMBOL")
head(mapped_go)

By default, `mapIds` will return the first match. If you really want all of the GO terms, you can specify the `multiVals` argument. Here's the options for `multiVals`:

first:

    This value means that when there are multiple matches only the 1st thing that comes back will be returned. This is the default behavior
list:

    This will just returns a list object to the end user
filter:

    This will remove all elements that contain multiple matches and will therefore return a shorter vector than what came in whenever some of the keys match more than one value
asNA:

    This will return an NA value whenever there are multiple matches
CharacterList:

    This just returns a SimpleCharacterList object
FUN:

    You can also supply a function to the multiVals argument for custom behaviors. The function must take a single argument and return a single value. This function will be applied to all the elements and will serve a 'rule' that for which thing to keep when there is more than one element. So for example this example function will always grab the last element in each result:  last <- function(x){x[[length(x)]]} 


Let's specify that we want `multiVals="list"`

In [ ]:
mapped_go <- mapIds(orgdb, keys=housekeeping, 
       column="GO", 
       keytype="SYMBOL",
       multiVals="list")
head(mapped_go)

## TxDB Objects

One of the other options in AnnotationHub is`TxDb`. They are also based on the AnnotationDb class and use similar methods.

A TxDb object connects a set of genomic coordinates to transcript-oriented features. It also contains feature IDs for transcripts and genes so TxDb objects can be used to link gene IDs and transcipt IDs.
Let's work with the human TxDb object:

In [ ]:
AnnotationHub::query(ah, pattern = c("Homo sapiens", "TxDb", "hg19"))

We can query the AnnotationHub and specify which record we'd like to use:

In [ ]:
txdb <- AnnotationHub::query(ah, pattern = c("Homo sapiens", "TxDb", "hg19"))[['AH52258']]

In [ ]:
txdb

Just like how we did with the OrgDb, we can look at what keytypes are available to us

In [ ]:
keytypes(txdb)

We can also use `select` in a similar way:

In [ ]:
select(txdb, keys = c("2597"), columns=c("TXNAME", "TXID", "CDSNAME"), keytype="GENEID")

Or `mapIds`

In [ ]:
mapIds(txdb, keys = c("2597"), column="TXNAME", keytype="GENEID", multiVals="list")

We can look at all the transcripts available in the txdb using the `transcripts()` function:

In [ ]:
transcripts(txdb)

We get back a GRanges object the location of each transcript, as well as its `tx_name` and `tx_id`. GRanges objects are just a way to show genomic locations (or Genomic Ranges) (https://www.bioconductor.org/packages/release/bioc/vignettes/GenomicRanges/inst/doc/GenomicRangesIntroduction.html).           

              
We can also look at `exons()`, `cds()`, `genes()` and `promoters()`.         
You can also look at transcripts grouped by the genes that they are associated with:

In [ ]:
txby <- transcriptsBy(txdb, by="gene")

In [ ]:
txby

Similar functions include `exonsBy()`, `cdsBy()`, `intronsByTranscript()`, `fiveUTRsByTranscript()`, and `threeUTRsByTranscript()`. 

We can also use `seqlevelsStyle` function  (exported from `GenomeInfoDb`) to get the current seqlevels style of an object and to rename its seqlevels according to a given style. 

In [ ]:
seqlevelsStyle(txdb)
seqinfo(txdb)

We can convert to 'NCBI' style:

In [ ]:
seqlevelsStyle(txdb) <- "NCBI"
seqinfo(txdb)

We can see what styles are supported using `genomeStyles`.

In [ ]:
head(genomeStyles("Homo_sapiens"))

Let's convert back to `UCSC` format:

In [ ]:
seqlevelsStyle(txdb) <- "UCSC"

You could filter the object to only look at a particular chromosome if you wanted to:

In [ ]:
seqlevels(txdb) <- "chr15"

# GenomicRanges

We introduced GRanges objects in the previous section about TxDb objects. GRanges objects are a series of genomic ranges with a start and end location on the genome. It can be used to store the location of genomic features such as contiguous binding sites, transcripts, and exons. These objects can be created by using the GRanges constructor function. For example:

In [27]:
gr1 <- GRanges(
    seqnames = Rle(c("chr1", "chr2", "chr1", "chr3"), c(1, 3, 2, 4)),
    ranges = IRanges(101:110, end = 111:120, names = head(letters, 10)),
    strand = Rle(strand(c("-", "+", "*", "+", "-")), c(1, 2, 2, 3, 2)),
    score = 1:10,
    GC = seq(1, 0, length=10))
gr1

GRanges object with 10 ranges and 2 metadata columns:
    seqnames    ranges strand |     score        GC
       <Rle> <IRanges>  <Rle> | <integer> <numeric>
  a     chr1   101-111      - |         1  1.000000
  b     chr2   102-112      + |         2  0.888889
  c     chr2   103-113      + |         3  0.777778
  d     chr2   104-114      * |         4  0.666667
  e     chr1   105-115      * |         5  0.555556
  f     chr1   106-116      + |         6  0.444444
  g     chr3   107-117      + |         7  0.333333
  h     chr3   108-118      + |         8  0.222222
  i     chr3   109-119      - |         9  0.111111
  j     chr3   110-120      - |        10  0.000000
  -------
  seqinfo: 3 sequences from an unspecified genome; no seqlengths

You can also build them from data frames, specifying the start and end site and indicating that you'd like to keep the extra metadata columns. For example:

In [28]:
seqnames <- c("chr1", "chr2", "chr3", "chr4", "chr5")
start <- c(100, 2150, 3200, 4250, 5300)
end <- c(1000, 15000, 25000, 30000, 40000)
strand <- c('-', '-', '+', '+', '+')
metadata <- c('.10', '.20', '.30', '.40', '.50')
df <- data.frame(seqnames, start, end, strand, metadata)
gr2 <- makeGRangesFromDataFrame(df, 
                              seqnames.field = 'seqnames',
                              start.field = 'start',
                              end.field = 'end',
                              strand.field = 'strand',
                              keep.extra.columns = TRUE)
gr2

GRanges object with 5 ranges and 1 metadata column:
      seqnames     ranges strand |    metadata
         <Rle>  <IRanges>  <Rle> | <character>
  [1]     chr1   100-1000      - |         .10
  [2]     chr2 2150-15000      - |         .20
  [3]     chr3 3200-25000      + |         .30
  [4]     chr4 4250-30000      + |         .40
  [5]     chr5 5300-40000      + |         .50
  -------
  seqinfo: 5 sequences from an unspecified genome; no seqlengths

There's several ways you can adjust or extend the ranges -- see https://bioconductor.org/packages/release/bioc/html/GenomicRanges.html. We'll discuss the `promoters` function here, but there's many, many more options.

`promoters` returns an object of the same type and length as x containing promoter ranges. Promoter ranges extend around the transcription start site (TSS) which is defined as start(x). The upsteam and downstream arguments define the number of nucleotides in the 5' and 3' direction, respectively. The full range is defined as,

(start(x) - upstream) to (start(x) + downstream - 1).

Ranges on the * strand are treated the same as those on the + strand. When no seqlengths are present in x, it is possible to have non-positive start values in the promoter ranges. This occurs when (TSS - upstream) < 1. In the equal but opposite case, the end values of the ranges may extend beyond the chromosome end when (TSS + downstream + 1) > 'chromosome end'. When seqlengths are not NA the promoter ranges are kept within the bounds of the defined seqlengths.

In [29]:
gr_promoters <- promoters(gr2, upstream=1000, downstream=200)
gr_promoters

GRanges object with 5 ranges and 1 metadata column:
      seqnames      ranges strand |    metadata
         <Rle>   <IRanges>  <Rle> | <character>
  [1]     chr1    801-2000      - |         .10
  [2]     chr2 14801-16000      - |         .20
  [3]     chr3   2200-3399      + |         .30
  [4]     chr4   3250-4449      + |         .40
  [5]     chr5   4300-5499      + |         .50
  -------
  seqinfo: 5 sequences from an unspecified genome; no seqlengths

We can combine this with the `rtracklayer` export function to make a GTF file

In [30]:
rtracklayer::export(gr_promoters, 'gr_promoters.gtf')

`rtracklayer` supports importing and exporting GRanges objects to or from many common bioinformatics formats -- see  https://bioconductor.org/packages/release/bioc/html/rtracklayer.html

There are also many functions in `GenomicRanges` to compare two GRanges objects -- we will talk about using `distancetoNearest`, but again there are many, many options that we aren't touching on.

distanceToNearest: Returns the distance for each range in x to its nearest neighbor in the subject.


In [ ]:
dist_to_nearest <- distanceToNearest(x = gr2, subject = gr1)
dist_to_nearest

`distanceToNearest` returns the indices of the query (x) and subject. We can pull out the the query and subject indices like this:

In [ ]:
query_i <- as.matrix(dist_to_nearest)[,1]
subject_i <- as.matrix(dist_to_nearest)[,2]
query_i
subject_i

Then use those indices to find the ranges and their distances:

In [ ]:
gr2_dist <- gr2[c(query_i),]
gr1_dist <- gr1[c(subject_i),]
gr2_dist
gr1_dist

Then pull it all together, converting the GRanges objects to data frames, giving them unique column names, and then making a new data frame with all the information about the ranges being compared and their distances.

In [ ]:
gr1_dist <- data.frame(gr1_dist)
gr2_dist <- data.frame(gr2_dist)

colnames(gr1_dist) <- paste0(colnames(gr1_dist), '_subject')
colnames(gr2_dist) <- paste0(colnames(gr2_dist), '_query')

out_df <- cbind(data.frame(gr1_dist), 
                data.frame(gr2_dist), 
                data.frame(dist_to_nearest))
out_df

# BSGenome

BSGenome is one option if you want to use R to search for actual sequence data. BSGenomes are `Biostrings-based` genomes, meaning that they use the package `BioStrings` to organize the data and facilitate access (https://bioconductor.org/packages/release/bioc/html/Biostrings.html). 

We can see which genomes are available:

In [ ]:
head(available.genomes())

You can load `BSgenome.Hsapiens.UCSC.hg19` or `Hsapiens` into the environment (we did this at the start of the notebook), and quickly confirm txdb and Hsapiens BSGenome are the same genome assembly (they are both hg19).

In [ ]:
Hsapiens
txdb

We can extract the exon ranges from `txdb` grouped by transcript:

In [ ]:
transcripts <- exonsBy(txdb, by="tx", use.names=TRUE)

Then we can extract the transcript sequences from the genome (we'll just use the first transcript to make it faster).

In [ ]:
tx_seqs <- extractTranscriptSeqs(Hsapiens, transcripts[1])

Then we can look and see that we have a `DNAStringSet` as the output -- the sequences.

In [ ]:
tx_seqs

# Ontology Analysis

Once we are at the step where we have genes that are differentially expressed, we can see if there is any enrichment in any functional gene groups. Two commonly used methods to look for enrichment are overrepresentation analysis (ORA) or gene set enrichment analysis (GSEA).          
- **Over Representation Analysis (ORA)** looks for functions or processes that are over-represented (= enriched) in an experimentally-derived gene list. The background used by default is all of the genes that have an annotation. This will find genes where the difference is large, but will not detect a situation where the difference is small but coordinated across a set of genes.      

- **Gene Set Enrichment (GSEA)** aggregates per-gene statistics across genes in a set. It takes a ranked list of genes and determines whether members of a gene set are randomly distributed throughout that list or if they are found primarily at the top or bottom of the list. GSEA will calculate an enrichment score based on whether a gene set is over-represented at the top or bottom fo the list, estimate the significance of the enrichment, and adjust for multiple hypothesis testing.       

There are many packages for running these types of analyses ([gage](https://www.bioconductor.org/packages/release/bioc/html/gage.html), [EnrichmentBrowser](https://www.bioconductor.org/packages/release/bioc/html/EnrichmentBrowser.html)) and many of them will use similar approaches to test for enrichment. We will use [clusterProfiler](https://www.bioconductor.org/packages/release/bioc/html/clusterProfiler.html).          

We will use [gene ontologies](http://geneontology.org/docs/ontology-documentation/) to organize the genes into groups based on their role in an organism. Gene Ontology loosely organize genes into three hierarchical graphs that correspond to three large umbrella categories -- **Molecular Function, Cellular Component, and Biological Process**. You can read the formal descriptions of these categories in the documentation linked above. A quote from the documentation illustrates an example of how these categories are related:        

```
In an example of GO annotation, the gene product “cytochrome c” can be described by the molecular function oxidoreductase activity, the biological process oxidative phosphorylation, and the cellular component mitochondrial matrix.
```

We can use our previously made `orgdb` object to run the enrichment analysis on `res`, which is the `results` object from DESeq2 differential expression analysis run on the `airway` data. We are comparing the dexamethasone treatment conditions, comparing treated to untreated.       

We will use the functions `gseGO` and `enrichGO` from clusterProfiler.      

- `gseGO` is a GSEA method, it takes a order ranked geneList as input and uses a Kolmogorov Smirnov test to run Gene Set Enrichment Analysis (GSEA) [Subramanian et al. 2005](https://www.ncbi.nlm.nih.gov/pmc/articles/pmid/16199517/). GSEA is useful in scenarios where the fold changes are subtle but modules of genes are regulated in a coordinated way.    
- `enrichGO` is an ORA method and takes a list of genes (does not neet to be ranked) and uses Fisher's exact test with a hypergeometric distribution to run Enrichment Analysis [Boyle et al. 2004](https://academic.oup.com/bioinformatics/article/20/18/3710/202612).     


In [ ]:
#Might need to re-load some packages at this point:
library("AnnotationHub")
library("clusterProfiler")
ah <- AnnotationHub()
orgdb <- AnnotationHub::query(ah, pattern = c("Homo sapiens", "OrgDb"))[[1]]

The DOSE package comes with a pre-made `geneList` for us to work with. Let's pull that data down and make a set of genes with a fold change larger than 2.

In [ ]:
data(geneList, package="DOSE")
gene <- names(geneList)[abs(geneList) > 2]

Then we can run `enrichGO` which is an ORA method and takes the following arguments:

`gene`	a vector of entrez gene id.\
`OrgDb`	OrgDb\
`keyType`	keytype of input gene\
`ont`	One of "BP", "MF", and "CC" subontologies, or "ALL" for all three. (Biological Process, Molecular Function, Cellular Compartment\
`pvalueCutoff`	adjusted pvalue cutoff on enrichment tests to report\
`pAdjustMethod`	one of "holm", "hochberg", "hommel", "bonferroni", "BH", "BY", "fdr", "none"\
`universe`	background genes. If missing, the all genes listed in the database (eg TERM2GENE table) will be used as background.\
`qvalueCutoff`	qvalue cutoff on enrichment tests to report as significant. Tests must pass i) pvalueCutoff on unadjusted pvalues, ii) pvalueCutoff on adjusted pvalues and iii) qvalueCutoff on qvalues to be reported.\
`minGSSize`	minimal size of genes annotated by Ontology term for testing.\
`maxGSSize`	maximal size of genes annotated for testing\
`readable`	whether mapping gene ID to gene Name\
`pool`	If ont='ALL', whether pool 3 GO sub-ontologies\    

In [ ]:
ego <- enrichGO(gene          = gene,
                universe      = names(geneList),
                OrgDb         = orgdb,
                ont           = "CC",
                pAdjustMethod = "BH",
                pvalueCutoff  = 0.01,
                qvalueCutoff  = 0.05,
                readable      = TRUE)

Then we can visualize the output with a dotplot:

In [ ]:
dotplot(ego, showCategory = 5)

The size of the dot indicates how many members of the group are represented in the enrichment and the adjusted p-value is the Benjamini-Hochberg corrected p-value. `GeneRatio` is `k/n`, where for a given category (e.g. 'receptor regulator activity') `k` is the overlap of 'receptor regulator activity' genes in `gene_list` compared to all 'receptor regulator activity' genes in the org.db, where `n` is the overlap of all genes in `gene_list` compares to all genes in the org.db.

Then we can run `gseGO` which is a GSEA method. It takes the following arguments:\
`geneList`	order ranked geneList\
`ont`	one of "BP", "MF", and "CC" subontologies, or "ALL" for all three.\
`OrgDb`	OrgDb\
`keyType`	keytype of gene\
`exponent`	weight of each step\
`minGSSize`	minimal size of each geneSet for analyzing\
`maxGSSize`	maximal size of genes annotated for testing\
`eps`	This parameter sets the boundary for calculating the p value.\
`pvalueCutoff`	pvalue Cutoff\
`pAdjustMethod`	pvalue adjustment method\
`verbose`	print message or not\
`seed`	logical\
`by`	one of 'fgsea' or 'DOSE'       

In [ ]:
ggo <- gseGO(geneList     = geneList,
              OrgDb        = orgdb,
              ont          = "CC",
              minGSSize    = 100,
              maxGSSize    = 500,
              pvalueCutoff = 0.05,
              verbose      = FALSE)

Then we can visualize the output with a dotplot:

In [ ]:
dotplot(ggo, showCategory = 5)

We can also use `enrichKEGG` and `gseKEGG` to get similar tests using KEGG data instead of GO data.

In [ ]:
kk <- enrichKEGG(gene         = gene,
                 organism     = 'hsa',
                 pvalueCutoff = 0.05)
dotplot(kk, showCategory = 5)

In [ ]:
kk2 <- gseKEGG(geneList     = geneList,
               organism     = 'hsa',
               minGSSize    = 120,
               pvalueCutoff = 0.05,
               verbose      = FALSE)
dotplot(kk2, showCategory = 5)